#Ingestion CSV vers BigQuery - Version Minimaliste

**Objectif** : Exécuter simplement la requête LOAD DATA du Tutoriel 1

**Prérequis** :
- Table `lake-471013.lakehouse_employee_data.employees` créée (Tutoriel 1)
- Fichier `employees_5mb.csv` uploadé dans GCS
- Permissions BigQuery et GCS

In [1]:
# Installation et imports
!pip install google-cloud-bigquery -q

from google.cloud import bigquery
from google.colab import auth
import os

print("✅ Installation terminée")

✅ Installation terminée


In [2]:
# Authentification Google Cloud
print(" -- Authentification en cours...")
auth.authenticate_user()

# Configuration du projet
PROJECT_ID = "lake-471013"  # ⚠️ Modifiez si nécessaire
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID

print(f"✅ Authentifié sur le projet: {PROJECT_ID}")

 -- Authentification en cours...
✅ Authentifié sur le projet: lake-471013


In [3]:
# Configuration des paramètres
FULL_TABLE_ID = "lake-471013.lakehouse_employee_data.employees"
GCS_URI = "gs://lakehouse-bucket-20250903/employees_5mb.csv"

print(f"📁 Table cible: {FULL_TABLE_ID}")
print(f"📄 Fichier source: {GCS_URI}")

# Initialiser le client BigQuery
client = bigquery.Client(project=PROJECT_ID)
print("✅ Client BigQuery initialisé")

📁 Table cible: lake-471013.lakehouse_employee_data.employees
📄 Fichier source: gs://lakehouse-bucket-20250903/employees_5mb.csv
✅ Client BigQuery initialisé


In [4]:
# 🚀 REQUÊTE D'INGESTION PRINCIPALE
ingestion_query = f"""
-- Flux d'ingestion CSV vers BigQuery
-- Fichier source : employees
-- Table cible : employees_5mb.csv
-- Truncate avant bulk

TRUNCATE TABLE `lake-471013.lakehouse_employee_data.employees`;

LOAD DATA INTO `lake-471013.lakehouse_employee_data.employees`
(id INT64, nom STRING, prenom STRING, email STRING, age INT64, ville STRING,
 code_postal STRING, telephone STRING, salaire FLOAT64, departement STRING,
 date_embauche DATE, statut STRING, score FLOAT64, latitude FLOAT64,
 longitude FLOAT64, commentaire STRING, reference STRING, niveau STRING,
 categorie STRING, timestamp TIMESTAMP)
FROM FILES (
  format = 'CSV',
  field_delimiter = ';',
  skip_leading_rows = 1,
  uris = ['gs://lakehouse-bucket-20250903/employees_5mb.csv']
);
"""
print(f"📄 Source: {GCS_URI}")
print(f"📁 Destination: {FULL_TABLE_ID}")

📄 Source: gs://lakehouse-bucket-20250903/employees_5mb.csv
📁 Destination: lake-471013.lakehouse_employee_data.employees


In [5]:
# Vérification rapide de la table
try:
    table = client.get_table(FULL_TABLE_ID)
    print(f"✅ Table trouvée: {table.table_id}")
    print(f"📊 Lignes actuelles: {table.num_rows:,}")
except Exception as e:
    print(f"❌ Erreur: {str(e)}")
    print("💡 Assurez-vous que la table existe (voir Tutoriel 1)")

✅ Table trouvée: employees
📊 Lignes actuelles: 18,100


In [7]:
# Exécution de l'ingestion
try:
    # Compter les lignes avant
    count_before = client.query(f"SELECT COUNT(*) as count FROM `{FULL_TABLE_ID}`").result()
    rows_before = list(count_before)[0].count
    print(f"📊 Lignes avant ingestion: {rows_before:,}")

    # Lancer l'ingestion
    print("\n⏳ Ingestion en cours...")
    job = client.query(ingestion_query)
    job.result()  # Attendre la fin

    # Compter les lignes après
    count_after = client.query(f"SELECT COUNT(*) as count FROM `{FULL_TABLE_ID}`").result()
    rows_after = list(count_after)[0].count

    # Résumé
    new_rows = rows_after - rows_before
    print("\n" + "="*50)
    print("✅ INGESTION TERMINÉE AVEC SUCCÈS!")
    print("="*50)
    print(f"📈 Lignes avant:     {rows_before:,}")
    print(f"📈 Lignes après:     {rows_after:,}")
    print(f"📄 Fichier traité:   {GCS_URI.split('/')[-1]}")

except Exception as e:
    print(f"\n❌ ERREUR LORS DE L'INGESTION:")
    print(f"   {str(e)}")
    print("\n💡 Vérifications suggérées:")
    print("   • Le fichier CSV existe-t-il dans GCS ?")
    print("   • Les permissions sont-elles correctes ?")
    print("   • La table de destination existe-t-elle ?")

📊 Lignes avant ingestion: 18,100

⏳ Ingestion en cours...

✅ INGESTION TERMINÉE AVEC SUCCÈS!
📈 Lignes avant:     18,100
📈 Lignes après:     18,100
📄 Fichier traité:   employees_5mb.csv
